# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [10]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [11]:
ds_train = tf.data.Dataset.load("../data/ds_train_5/")
ds_val = tf.data.Dataset.load("../data/ds_val_5/")
ds_test = tf.data.Dataset.load("../data/ds_test_5/")

In [12]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3399187, 482548, 104452)

In [13]:
BATCH_SIZE = 1024
BLOCK_SIZE = 32

In [14]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [15]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [16]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(x[:,:,:-3].shape, x[:,:,-3].shape, x[:,:,-2].shape, x[:,:,-1].shape)
    print(y.shape)

(1024, 32, 9)
(1024, 32, 6) (1024, 32) (1024, 32) (1024, 32)
(1024, 1)


In [17]:
ys = []
for x, y in ds_train.batch(BATCH_SIZE):
    ys.append(y.numpy())
ys = np.concatenate(ys)
np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

array([2842099,  557088])

In [18]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 2842099, 557088
total = neg + pos
total, pos, neg

(3399187, 557088, 2842099)

In [19]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(32, n_embed // 9, name="embed_plate")
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_dd")
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_magtype")
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6, name="embed_continous")
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True, name="lstm1")
        self.lstm2 = tf.keras.layers.LSTM(n_embed, name="lstm2")
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu', name="pre_classifier")
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid', name="classifier")
        self.conc = tf.keras.layers.Concatenate(axis=-1, name="concatenate")

    def build_graph(self):
        x = [tf.keras.layers.Input(shape=(BLOCK_SIZE, 6), batch_size=None, name="input_continous"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_plate"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_dd"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_magtype")]
        return tf.keras.Model(inputs=x, outputs=self.call(x))

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1]
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [21]:
model = MyModel(135)

In [22]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

312886

In [23]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [24]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val, class_weight=class_weight)

Epoch 1/20


2024-01-19 11:40:48.530694: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 221 of 2000
2024-01-19 11:41:08.447511: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 658 of 2000
2024-01-19 11:41:28.441335: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1048 of 2000
2024-01-19 11:41:48.436016: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1509 of 2000
2024-01-19 11:42:08.424886: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1881 of 2000
2024-01-19 11:42:14.172193: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 11:42:17.622303: I tensorfl

3320/3320 [==============================] - 336s 68ms/step - loss: 0.5266 - accuracy: 0.7859 - recall: 0.6551 - precision: 0.4052 - val_loss: 0.5890 - val_accuracy: 0.6619 - val_recall: 0.5739 - val_precision: 0.3000
Epoch 2/20


2024-01-19 11:46:20.770162: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 188 of 2000
2024-01-19 11:46:30.805236: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 380 of 2000
2024-01-19 11:46:40.805280: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 574 of 2000
2024-01-19 11:46:50.814971: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 784 of 2000
2024-01-19 11:47:10.806063: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1231 of 2000
2024-01-19 11:47:30.769398: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a

3320/3320 [==============================] - 306s 63ms/step - loss: 0.4911 - accuracy: 0.8044 - recall: 0.6866 - precision: 0.4382 - val_loss: 0.5285 - val_accuracy: 0.7399 - val_recall: 0.5023 - val_precision: 0.3686
Epoch 3/20


...

3320/3320 [==============================] - 349s 78ms/step - loss: 0.2285 - accuracy: 0.8865 - recall: 0.9180 - precision: 0.6005 - val_loss: 1.7693 - val_accuracy: 0.7359 - val_recall: 0.4543 - val_precision: 0.3529
Epoch 19/20


2024-01-19 13:20:38.284469: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 227 of 2000
2024-01-19 13:20:48.318174: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 454 of 2000
2024-01-19 13:20:58.323944: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 682 of 2000
2024-01-19 13:21:18.304503: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1142 of 2000
2024-01-19 13:21:28.315228: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1372 of 2000
2024-01-19 13:21:48.319742: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take 

3320/3320 [==============================] - 339s 76ms/step - loss: 0.2154 - accuracy: 0.8921 - recall: 0.9258 - precision: 0.6131 - val_loss: 1.7443 - val_accuracy: 0.7501 - val_recall: 0.4623 - val_precision: 0.3761
Epoch 20/20


2024-01-19 13:26:17.193320: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 228 of 2000
2024-01-19 13:26:27.217611: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 458 of 2000
2024-01-19 13:26:37.220293: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 688 of 2000
2024-01-19 13:26:57.228217: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1139 of 2000
2024-01-19 13:27:17.206078: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 1594 of 2000
2024-01-19 13:27:34.955216: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


3320/3320 [==============================] - 339s 76ms/step - loss: 0.2029 - accuracy: 0.8977 - recall: 0.9339 - precision: 0.6259 - val_loss: 1.8379 - val_accuracy: 0.7610 - val_recall: 0.4281 - val_precision: 0.3878


In [25]:
model.evaluate(ds_test)

103/103 [==============================] - 7s 71ms/step - loss: 0.5006 - accuracy: 0.8293 - recall: 0.6659 - precision: 0.3906


[0.5005568861961365,
 0.8292517066001892,
 0.6659225821495056,
 0.3906348645687103]

In [26]:
model.save("../models/model_v8_cw.keras")

In [27]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v8_cw.csv", index=False)